In [5]:
import sys
import os

os.chdir(os.path.abspath(".."))
sys.path.append(os.path.abspath(".."))

In [6]:
import os
import logging
from datetime import datetime, timedelta

import requests
import pandas as pd
from sqlalchemy import create_engine, text
from dotenv import load_dotenv

# .envファイルから環境変数を読み込む
load_dotenv()
EDINET_API_KEY = os.getenv('EDINET_API_KEY')
DATABASE_URL = os.getenv('DATABASE_URL')

# PostgreSQLに接続するSQLAlchemyエンジンを作成
engine = create_engine(DATABASE_URL)

# ロギング設定: 書類取得成功時のみINFOレベルで表示し、それ以外は極力抑制
logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s')
logging.getLogger("requests").setLevel(logging.WARNING)  # requestsの詳細ログは抑制

# パラメータとなる日付範囲を設定（必要に応じて変更可能）
START_DATE = datetime(2017, 1, 1)
END_DATE = datetime(2025, 12, 31)

In [7]:
# EDINET APIエンドポイントとヘッダの設定
EDINET_LIST_ENDPOINT = "https://api.edinet-fsa.go.jp/api/v2/documents.json"
API_HEADERS = {"Ocp-Apim-Subscription-Key": EDINET_API_KEY}

# 結果を格納するDataFrameを準備
all_documents_df = pd.DataFrame()

current_date = START_DATE
while current_date <= END_DATE:
    date_str = current_date.strftime("%Y-%m-%d")
    params = {"date": date_str, "type": 2}
    try:
        response = requests.get(EDINET_LIST_ENDPOINT, params=params, headers=API_HEADERS, timeout=10)
    except Exception as e:
        logging.error(f"API呼び出し失敗: 日付{date_str} - エラー: {e}")
        current_date += timedelta(days=1)
        continue
    
    if response.status_code != 200:
        # ステータスコード200以外の場合はメッセージを出力してスキップ
        logging.error(f"API呼び出し失敗: 日付{date_str} - ステータスコード: {response.status_code}")
        current_date += timedelta(days=1)
        continue
    
    data = response.json()
    results = data.get("results", [])
    if not results:
        # 提出書類がない場合
        current_date += timedelta(days=1)
        continue
    
    # DataFrameに追加
    day_df = pd.DataFrame(results)
    all_documents_df = pd.concat([all_documents_df, day_df], ignore_index=True)
    
    current_date += timedelta(days=1)

# 取得した書類件数を確認
print(f"取得した書類件数: {len(all_documents_df)}")
all_documents_df.head(5)

取得した書類件数: 768886


,seqNumber,docID,edinetCode,secCode,JCN,filerName,fundCode,ordinanceCode,formCode,docTypeCode,...,opeDateTime,withdrawalStatus,docInfoEditStatus,disclosureStatus,xbrlFlag,pdfFlag,attachDocFlag,englishDocFlag,csvFlag,legalStatus
0,1,S1009555,None,None,None,None,None,None,None,None,...,None,0,0,0,0,0,0,0,0,0
1,2,S1009DWD,None,None,None,None,None,None,None,None,...,None,0,0,0,0,0,0,0,0,0
2,3,S10099B4,None,None,None,None,None,None,None,None,...,None,0,0,0,0,0,0,0,0,0
3,4,S1009E8H,None,None,None,None,None,None,None,None,...,None,0,0,0,0,0,0,0,0,0
4,5,S1009DWK,None,None,None,None,None,None,None,None,...,None,0,0,0,0,0,0,0,0,0


In [8]:
# companiesテーブルから会社IDとEDINETコードを取得
companies_df = pd.read_sql("SELECT company_id, edinet_code FROM companies", con=engine)
company_map = {row['edinet_code']: row['company_id'] for _, row in companies_df.iterrows()}

# EDINETコードの集合（高速なフィルタリング用）
edinet_codes_set = set(company_map.keys())

# 取得した書類データから、companiesテーブルに存在するEDINETコードのみ抽出
filtered_docs_df = all_documents_df[all_documents_df['edinetCode'].isin(edinet_codes_set)].copy()

# 書類種別コードで報告書系（年次・四半期・半期・臨時および訂正）に該当するもののみ残す
target_doc_types = {"120","130","140","150","160","170","180","190"}
filtered_docs_df = filtered_docs_df[ filtered_docs_df['docTypeCode'].isin(target_doc_types) ].copy()

print(f"対象企業の提出書類件数: {len(filtered_docs_df)}")
filtered_docs_df[['docID','edinetCode','docTypeCode','submitDateTime','docDescription']].head(10)

対象企業の提出書類件数: 39242


,docID,edinetCode,docTypeCode,submitDateTime,docDescription
426,S1009ENA,E03248,140,2017-01-06 10:31,四半期報告書－第38期第3四半期(2016/09/01－2016/11/30)
875,S1009F48,E02925,140,2017-01-11 09:22,四半期報告書－第32期第3四半期(2016/09/01－2016/11/30)
990,S1009F38,E00323,140,2017-01-11 11:10,四半期報告書－第20期第2四半期(2016/09/01－2016/11/30)
1084,S1009F1K,E21035,140,2017-01-11 15:02,四半期報告書－第9期第3四半期(2016/09/01－2016/11/30)
1275,S1009FE3,E03344,140,2017-01-12 10:08,四半期報告書－第35期第3四半期(2016/09/01－2016/11/30)
1291,S1009FKM,E03153,140,2017-01-12 10:38,四半期報告書－第60期第3四半期(2016/09/01－2016/11/30)
1297,S1009FD1,E03074,140,2017-01-12 10:44,四半期報告書－第58期第1四半期(2016/09/01－2016/11/30)
1319,S1009FGK,E32736,140,2017-01-12 11:31,四半期報告書－第13期第3四半期(2016/09/01－2016/11/30)
1327,S1009EZH,E03013,140,2017-01-12 12:13,四半期報告書－第151期第3四半期(2016/09/01－2016/11/30)
1383,S1009FBG,E32238,140,2017-01-12 15:08,四半期報告書－第18期第3四半期(2016/09/01－2016/11/30)


In [9]:
from zipfile import ZipFile
from io import BytesIO
from tqdm.notebook import tqdm
import os
import requests
import pandas as pd

# CSV保存用のベースディレクトリ
BASE_DIR = "data/raw/edinet_csv_data"
os.makedirs(BASE_DIR, exist_ok=True)

saved_files = []
skipped_count = 0
error_count = 0

for idx, row in tqdm(filtered_docs_df.iterrows(), total=len(filtered_docs_df), desc="CSV取得中"):
    doc_id = row['docID']
    edinet_code = row['edinetCode']
    doc_type = row['docTypeCode']
    submit_datetime = row['submitDateTime']
    csv_flag = row.get('csvFlag', "0")

    if str(csv_flag) != "1":
        skipped_count += 1
        continue

    submit_year = pd.to_datetime(submit_datetime).year
    save_dir = os.path.join(BASE_DIR, str(submit_year), edinet_code)
    os.makedirs(save_dir, exist_ok=True)

    doc_url = f"https://api.edinet-fsa.go.jp/api/v2/documents/{doc_id}?type=5"
    try:
        res = requests.get(doc_url, headers=API_HEADERS, timeout=10)
        res.raise_for_status()
    except Exception:
        error_count += 1
        continue

    with ZipFile(BytesIO(res.content)) as zip_file:
        csv_filename = None
        for name in zip_file.namelist():
            name_l = name.lower()
            if doc_type in ["120", "130"] and "asr" in name_l:
                csv_filename = name
                break
            elif doc_type in ["140", "150"] and "qsr" in name_l:
                csv_filename = name
                break
            elif doc_type in ["160", "170"] and "hsr" in name_l:
                csv_filename = name
                break
            elif doc_type in ["180", "190"] and "esr" in name_l:
                csv_filename = name
                break

        if csv_filename is None:
            for name in zip_file.namelist():
                if name.lower().endswith(".csv"):
                    csv_filename = name
                    break

        if csv_filename is None:
            skipped_count += 1
            continue

        save_path = os.path.join(save_dir, f"{doc_id}.csv")
        with zip_file.open(csv_filename) as zf, open(save_path, 'wb') as f_out:
            f_out.write(zf.read())

        saved_files.append(save_path)

print("✅ 完了:")
print(f" - 成功数     : {len(saved_files)}")
print(f" - スキップ数 : {skipped_count}")
print(f" - エラー数   : {error_count}")

CSV取得中:   0%|          | 0/39242 [00:00<?, ?it/s]

✅ 完了:
 - 成功数     : 39174
 - スキップ数 : 68
 - エラー数   : 0


In [11]:
# edinet_documentsテーブルへのUPSERT処理
insert_count = 0
update_count = 0

for idx, row in filtered_docs_df.iterrows():
    doc_id = row['docID']
    edinet_code = row['edinetCode']
    # companiesテーブルに存在しないコードは対象外だが、一応ガード
    if edinet_code not in company_map:
        continue
    company_id = company_map[edinet_code]
    doc_type_code = row['docTypeCode']
    submit_datetime = pd.to_datetime(row['submitDateTime'])
    submit_date = submit_datetime.strftime("%Y-%m-%d %H:%M:%S")  # datetime文字列
    fiscal_year = submit_datetime.year  # 簡易的に提出年を会計年度とする
    description = row.get('docDescription') or ''
    
    # ローカルCSVパスを取得: ダウンロード済みリストからdocIDを検索
    # ダウンロードされなかった場合はNULLとする
    local_csv_path = None
    for path in saved_files:
        if f"{doc_id}.csv" in path:
            local_csv_path = path
            break
    
    # UPSERTクエリの組み立て
    upsert_sql = text("""
        INSERT INTO edinet_documents 
            (company_id, edinet_code, doc_id, doc_type_code, submit_date, fiscal_year, description, local_csv_path)
        VALUES 
            (:company_id, :edinet_code, :doc_id, :doc_type_code, :submit_date, :fiscal_year, :description, :local_csv_path)
        ON CONFLICT (doc_id)
        DO UPDATE SET 
            company_id = EXCLUDED.company_id,
            edinet_code = EXCLUDED.edinet_code,
            doc_type_code = EXCLUDED.doc_type_code,
            submit_date = EXCLUDED.submit_date,
            fiscal_year = EXCLUDED.fiscal_year,
            description = EXCLUDED.description,
            local_csv_path = EXCLUDED.local_csv_path;
    """)
    params = {
        "company_id": company_id,
        "edinet_code": edinet_code,
        "doc_id": doc_id,
        "doc_type_code": doc_type_code,
        "submit_date": submit_date,
        "fiscal_year": fiscal_year,
        "description": description,
        "local_csv_path": local_csv_path
    }
    with engine.begin() as conn:
        result = conn.execute(upsert_sql, params)
        # PostgreSQLのINSERT...ON CONFLICTではResultProxyから挿入/更新件数が取得しづらいので、
        # ここでは単純に全件を挿入したものとみなす
    insert_count += 1

print(f"`edinet_documents` に登録（更新含む）した件数: {insert_count}")

`edinet_documents` に登録（更新含む）した件数: 39242
